# 利用Python 比對兩個敦煌XML專案的缺字使用情況

## 1. 需求函式庫

In [73]:
from lxml import etree
import re
import os
import requests
import operator


## 2.設定參數

In [107]:
# XML 資料夾 與 檔名
XMLDIR="data"
XMLFILEs = [
  "LIN-pomo-S-3491.xml",
  "LIN-pomo-P-2187.xml"
  # "LIN3-Huanxiguowangyuan-P-3375v.xml"
  ]

# 輸出檔資料夾
OUTDIR="out"
# 輸出檔檔名
gcOUTFILE="g-out.html"
# 組合輸出檔位置
gcOUTPATH = os.path.join(".", OUTDIR, gcOUTFILE)

# 圖檔資料夾
IMGDIR = "imgs"


## 3. 相關副函式
### 3.1 Orig 擷取函式

In [10]:
def getORIG(xmlpath):
  # 回傳用
  origData ={}
  
  # 解析XML Tree
  xmltree = etree.parse(xmlpath)
  root = xmltree.getroot()
  
  # 抓取所有origNode
  origNodes = xmltree.xpath(
      '//tei:orig', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
  
  # 逐一處理OrigNode
  for node in origNodes:
    reg = node.get("reg")

    # 有子節點，應該要是g
    if len(node) >0 and node[0].tag == "{http://www.tei-c.org/ns/1.0}g":
      key = node[0].get("ref")
      type ="g"
      data = node[0]
      # print("{}(g-ref)".format(key))
    elif len(node) > 0:
      key = etree.tostring(node, method="text",
                           encoding="utf-8", with_tail=False).decode("utf-8")
      type="text"
      data=key
      # print("{}(to-string)".format(key))
    else:
      key = node.text
      # print("{}(node-text)".format(key))
      type ="text"
      data = key
    
    # 更新回傳資料
    # 結構為： origData={reg:
    #                    {key:{"data":data,"type":type, "count":c}, ..}
    #                  }
    if not (reg in origData):
      origData[reg] = {key:{"data":data,"type":type,"count":1}}
    else:
      if not(key in origData[reg]):
        origData[reg][key] = {"data": data, "type": type, "count": 1}
      else:
        origData[reg][key]["count"] +=1

  return origData


### 3.2 orig 內容列印

In [11]:
def orig_to_htmlstr(orig):
  s = ""
  
  for key in orig:
     imgstr = ""
     if orig[key]["type"] == "g":
        imgstr = "<img src='{}'>".format(os.path.join("..",IMGDIR,key[1:]+".png"))
     s += "<li>{}({}){}[{}]</li>".format(key, orig[key]
                                         ["type"], imgstr, orig[key]["count"])

  return "<ol>{}</ol>".format(s)


### 3.3 比對一組orig的keys相似度

In [ ]:
def orig_similarity(origs):
  """
  情況：
    origs 是一個list，裡面每一格就是orig的字型info
    origData[reg] = [{key1:{"data": data, "type": type, "count": 1},key2:{...}},...]
  計算方式：
    1.每一格抓取各自的keys 成為 set1, set2, set3... setK, 個數表示為 N1, N2, N3,...NK
    2.計算 intersect(set1, set2, set3, ... setK), 個數為 NS
    3.相似度為 K*NS / (N1+N2+N3...NK)
  """
  setCount = 0  # 計算總數用
  keys_intersect = None # 計算交集 set
  for origKeyDict in origs:
    setCount += len(origKeyDict) # 計算總數 NS
    if keys_intersect == None:
      keys_intersect = set(origKeyDict.keys())  # 初次必須抓取完整交集 set
    else:
      keys_intersect = keys_intersect.intersection(
          set(origKeyDict.keys()))  # 計算交集 set
          
  # 相似度為 K*NS / (N1+N2+N3...NK)
  return len(keys_intersect)*len(origs)/setCount

  


### 3.4 orig 比對與結果匯出

In [101]:
def origComp(origData, cmpOrder=[]):
  """
    1. 目的為比對不同來源的 orig Data
    2. cmpOrder 所指定的比對範圍，不指定就是全部比
    3. cmpOrder 指定方式，就是用origData 的key

  """

  rHTML = """
  <!DOCTYPE html>
  <html>
    <head>
      <title>缺字比較表</title><meta charset = "UTF-8"/><link rel="stylesheet" href="mystyle.css">
    </head>
    <body>
      <h1>缺字比較表</h1><h2>比對結果彙總</h2>
      <table>{}</table>
      <h2>細節內容</h2>
      <table>
        <thead>{}</thead><tbody>{}</tbody>
      </table>
    </body>
  </html>
  """  # 回傳字串

  # 處理 comOrder(比較順序) 的預設值
  if len(cmpOrder) == 0:
    cmpOrder = list(origData.keys())

  # 產生比較表的表頭字串

  rTableHeadStr="<tr><th>No.</th><th>Reg</th>{}<th>相似程度</th></tr>".format("".join(["<th>"+c+"</th>" for c in cmpOrder]))
  print(rTableHeadStr)

  # all_regs_union = set() #所有regs 的 union
  # for cmpo in cmpOrder:
  #   all_regs_union = all_regs_union.union(set(origData[cmpo].keys()))

  # 準備比較對象，取 intersection
  all_regs_intersect = None  # 所有regs 的 intersect
  for cmpo in cmpOrder:
    if all_regs_intersect == None: # 取intersect 的第一步，必須全載入第一組。
      all_regs_intersect = set(origData[cmpo].keys())
    else:
      all_regs_intersect = all_regs_intersect.intersection(set(origData[cmpo].keys()))

  # 產生比較表的內容字串，因為需要排序，將暫時丟到矩陣中
  rTableRowStrs = []

  # 用來計算各種狀況的比例
  counts = {"fM": 0, "pM": 0, "uM": 0}  #fullMatch, partialMatch, unMatch


  # 逐一抓出重疊的regs 出來輸出與比對
  for reg in all_regs_intersect:
    rRegNoStr = "<td>{}</td>" # 第一欄 No. 因為需要排序後產生，暫時空著
    rRegHeadStr = "<td>{}</td>".format(reg)  # 第二欄 reg
    
    rRegXmlsStr = ""  # 紀錄不同xml內，相同Reg的內容，一個Reg內的內容，便經常會有多值，內容將產生為ol+li
    for cmpo in cmpOrder:
      if reg in origData[cmpo]:
        # Reg的內容，經常會有多值，內容將產生為ol+li
        rRegXmlsStr += "<td>{}</td>".format(orig_to_htmlstr(origData[cmpo][reg]))
      else:
        # 若為 intersction 則永遠不會到這裡
        rRegXmlsStr += "<td></td>"

    # 計算相似度，並換為對應字串
    osim=orig_similarity([origData[cmpo][reg] for cmpo in cmpOrder])
    if osim == 1:
      counts["fM"]+=1
      rRegXmlsStr += "<td class='tBlue'>完全相同(100%)</td>"
    elif osim==0:
      counts["uM"] += 1
      rRegXmlsStr += "<td class='tRed'>完全不同(0%)</td>"
    else:
      counts["pM"] += 1
      rRegXmlsStr += "<td>部分相同</td>"

    # 將no. 的空欄，多個reg字串與osim值，以單一封裝來紀錄, 並保留osim 等待排序
    rTableRowStrs.append([rRegNoStr+rRegHeadStr+rRegXmlsStr, osim])

  # 利用第2欄位(osim值)，重新排序
  rTableRowStrs = sorted(rTableRowStrs, key=operator.itemgetter(1), reverse=True)
  # 先把 No. 值 以 i+1 封入後，再放入整體HTML索引
  rTableBodyStr = "".join(["<tr>{}</tr>".format(rowStr[0].format(i+1)) for i, rowStr in enumerate(rTableRowStrs)])
  

  # 產生index Table, 計算簡單，但格式複雜。
  rIndexTableStr = """
          <tr><th>分類</th><th>總數</th><th>比例</th></tr>
          <tr><td class='tBlue'>完全相同(100%)</td><td class='bold'>{1}</td><td>{4:.2f}%</td></tr>
          <tr><td>部分相同</td><td class='bold'>{2}</td><td>{5:.2f}%</td></tr>
          <tr><td class='tRed'>完全不同(0%)</td><td class='bold'>{3}</td><td>{6:.2f}%</td></tr>
          <tr><td>小計</td><td>{0}</td><td>-</td>""".format(
      len(all_regs_intersect), counts["fM"], counts["pM"], counts["uM"], 
      counts["fM"]/len(all_regs_intersect)*100, 
      counts["pM"]/len(all_regs_intersect)*100, 
      counts["uM"]/len(all_regs_intersect)*100)

  # 最後統整整個 HTML 內容，並送出。
  return rHTML.format(rIndexTableStr,rTableHeadStr, rTableBodyStr)


## 4. 主程式

In [108]:
ALLOrigData ={}
for xmlfile in XMLFILEs:
  print ("======== {} ========".format(xmlfile))
  origs=getORIG(os.path.join(XMLDIR,xmlfile))
  ALLOrigData[xmlfile]=origs

htmlstr = origComp(ALLOrigData)

with open(gcOUTPATH,"w") as ofile:
  ofile.write(htmlstr)
  print("輸出結果至"+gcOUTPATH)


======== LIN-pomo-S-3491.xml ========
======== LIN-pomo-P-2187.xml ========
======== LIN3-Huanxiguowangyuan-P-3375v.xml ========
<tr><th>No.</th><th>Reg</th><th>LIN-pomo-S-3491.xml</th><th>LIN-pomo-P-2187.xml</th><th>LIN3-Huanxiguowangyuan-P-3375v.xml</th><th>相似程度</th></tr>
輸出結果至./out/g-out.html
